In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import math
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import TensorDataset, DataLoader
from torch.nn import TransformerEncoder, TransformerEncoderLayer
import torch.nn.functional as F
from matplotlib import pyplot as plt
import random
device = torch.device("cuda:0")

## Data pre-processing ##

In [ ]:
def preprocess_data(df, columns_with_std_zero):
    X = df.drop(columns=['Energy','w'] + columns_with_std_zero)
    y = df['Energy']
    return X, y

# Define loss function
def MAPELoss(output, target):
    return torch.mean(torch.abs((target - output) / target))

def calculate_mape(y_true, y_pred):
    y_true = y_true.cpu().detach().numpy()
    y_pred = y_pred.cpu().detach().numpy()
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def calculate_wmape(y_true,y_pred, weights):
    return np.sum(weights * np.abs(y_true - y_pred)) / np.sum(weights)

def WMAPELoss(y_pred, y_true, weights):
    return torch.sum(weights * torch.abs(y_true - y_pred)) / torch.sum(weights)

# define model
class LinearBlock(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearBlock, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)
        self.bn = nn.BatchNorm1d(output_dim)
        self.act = nn.LeakyReLU()
        
    def forward(self, x):
        x = self.fc(x)
        x = self.bn(x)
        x = self.act(x)
        return x
    
class MyModel(nn.Module):
    def __init__(self, input_dim):
        super(MyModel, self).__init__()
        self.block1 = LinearBlock(input_dim, 512)
        # self.block2 = LinearBlock(1024, 512)
        self.block3 = LinearBlock(512, 256)
        self.block4 = LinearBlock(256, 128)
        self.block5 = LinearBlock(128, 64)
        self.fc_final = nn.Linear(64, 1)
        
    def forward(self, x):
        x = self.block1(x)
        # x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)
        x = self.fc_final(x)
        return x

def weight_init(m):
    if isinstance(m, torch.nn.Conv1d):
        torch.nn.init.kaiming_uniform_(m.weight, nonlinearity='relu')
        if m.bias is not None:
            torch.nn.init.zeros_(m.bias)
    elif isinstance(m, torch.nn.Linear):
        torch.nn.init.kaiming_uniform_(m.weight, nonlinearity='relu')
        if m.bias is not None:
            torch.nn.init.zeros_(m.bias)

In [ ]:
base_info = pd.read_csv('EnergyContest/BSinfo.csv')
cell_data = pd.read_csv('EnergyContest/CLdata.csv', parse_dates=['Time'])
energy_data = pd.read_csv('EnergyContest/ECdata.csv', parse_dates=['Time'])
power_consumption_prediction = pd.read_csv('EnergyContest/power_consumption_prediction.csv', parse_dates=['Time'])

duplicate_rows = cell_data.duplicated(subset=['Time', 'BS'])
print("Number of duplicate rows based on Time and BS:", duplicate_rows.sum())
cell_data = cell_data.drop_duplicates(subset=['Time', 'BS'])


Total = pd.merge(base_info,cell_data, on=['BS', 'CellName'], how='inner')
train_data = pd.merge(energy_data,Total, on=['Time', 'BS'], how='left')
test_data = pd.merge(power_consumption_prediction, Total, on=['Time', 'BS'], how='left')

combined_data = pd.concat([train_data, test_data], ignore_index=False)
combined_data['Time'] = pd.to_datetime(combined_data['Time']).dt.dayofyear*24+pd.to_datetime(combined_data['Time']).dt.hour
combined_data.drop(columns=['CellName'],inplace = True)
combined_data = pd.get_dummies(combined_data, columns=['Mode','RUType','Antennas','Bandwidth','Frequency','Time','BS'])

train_data = combined_data.iloc[:len(train_data)]
test_data = combined_data.iloc[len(train_data):]

columns_with_std_zero1 = train_data.columns[train_data.std() == 0].tolist()
print(len(columns_with_std_zero1))
print(columns_with_std_zero1)
print(len(train_data.columns[train_data.mean() == 0].tolist()))
columns_with_std_zero2 = train_data.columns[test_data.std() == 0].tolist()
print(len(columns_with_std_zero2))
print(columns_with_std_zero2)
columns_with_std_zero = list(set(columns_with_std_zero1+ columns_with_std_zero2))
print(len(columns_with_std_zero))

X,y = preprocess_data(train_data,columns_with_std_zero)
X_test,y_test = preprocess_data(test_data,columns_with_std_zero)

print(columns_with_std_zero)
print(X.shape[1])
print(X.shape[0])
print(X_test.shape[1])
print(X_test.shape[0])
print(Total.head())


## Train ##

In [ ]:
seed = 42
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

continue_train = 0
X_tensor = torch.Tensor(X.values).to(device)
y_tensor = torch.Tensor(y.values).unsqueeze(1).to(device)
X_data = TensorDataset(X_tensor, y_tensor)
X_loader = DataLoader(dataset=X_data, batch_size=512, shuffle=True)

if continue_train == 1:
    model = torch.load("./model_last_w1.pth")
else:
    model = MyModel(X.shape[1]).to(device)

criterion = MAPELoss
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.001)

# Train the model
num_epochs = 10000

best_mape = 100
best_epoch = 0

try:
    for epoch in range(num_epochs):
        running_loss = 0.0
        running_mape = 0.0
        model.train()  # Set the model to training mode
        for i, data in enumerate(X_loader, 0):
            inputs, labels = data
            optimizer.zero_grad()
            
            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            # print(loss)
            loss.backward()
            optimizer.step()

            # calculate running loss and MAPE
            running_loss += loss.item()*100
            # running_mape += calculate_mape(labels, outputs).item()
        print('Epoch %d, loss: %.3f' % (epoch + 1, running_loss / len(X_loader)))
        # print('Epoch %d, loss: %.3f, MAPE: %.3f' % (epoch + 1, running_loss / len(X_loader), running_mape / len(X_loader)))

        if best_mape > (running_loss / len(X_loader)):
            torch.save(model, "./model_best_w1.pth")
            best_mape = running_loss / len(X_loader)
            best_epoch = epoch + 1
        if  running_loss / len(X_loader) <= 0.4:
            break
    torch.save(model, "./model_last_w1.pth")
    print('Finished Training')
except KeyboardInterrupt:
    torch.save(model, "./model_last_w1.pth")
    print(f"Model saved to {'./model_last_w1.pth'}")


## Evaluate ##

In [ ]:
X_test_tensor = torch.Tensor(X_test.values).to(device)
y_test_tensor = torch.Tensor(y_test.values).unsqueeze(1).to(device)
test_data = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(dataset=test_data, batch_size=1, shuffle=False)

evaluate_flag = 1
if evaluate_flag == 1:
    model = torch.load("BestModel/best_w1.pth")
else:
    model = torch.load("./model_best_w1.pth")
    
model.eval()  # Set the model to evaluation mode
predictions = []
with torch.no_grad():  # We don't need gradients for prediction
    for i, data in enumerate(test_loader, 0):
        inputs, _ = data  # Ignore the weights
        outputs = model(inputs)
        predictions.extend(outputs.cpu().numpy())

predictions_numpy = np.array(predictions)
predictions_flatten = predictions_numpy.flatten()
predictions_series = pd.Series(predictions_flatten, name="PredictedEnergy")

test_data_new = pd.read_csv('EnergyContest/power_consumption_prediction.csv')
test_data_new['Energy'] = predictions_series
test_data_new['ID'] = test_data_new['Time'].astype(str) + "_" + test_data_new['BS'].astype(str)
test_data_new = test_data_new[['ID', 'Energy']]
test_data_new.to_csv('./power_consumption_prediction_w1.csv', index=False)